# Case 1 Part 1: Calculating Running Average and Standard Deviation

One of the analytics use case to be tackled is a real-time alerting system:

`Hourly consumption for a household is higher than 1 standard deviation of that household's historical mean consumption for that hour.`

To do that, I split the use case into three parts. 

First, read the data stream from the `readings_prepared` topic, group them by `house_id` and the **hour** of the `reading_ts`, and calculate the statistics (average and standard deviation) of that group. Everytime the value changes, or every small period of time, we persist the change inside a database / stream. Let's name the target store `alert_1_stats`. This first task is reflected on this notebook. 

The second part is reading the data stream from both `readings_prepared` and `alert_1_stats` and use them to detect anomalies, and store the anomalies in a persistent data store. 

The third task is a developign scheduled job to check that data store to see if there are anomalies and send alerts to the related parties. **The last 2 tasks will have their own notebook / code.**


## Setup

Import all the required libraries and set the stream configuration variables.

In [ ]:
import spark.implicits._
import java.sql.Timestamp
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.streaming.{GroupStateTimeout, OutputMode, GroupState}
import org.apache.spark.sql.streaming.Trigger

In [ ]:
val kafkaBootstrapServer = "kafka-m:9092"
val kafkaSourceTopic = "readings_prepared"
val kafkaTargetTopic = "alert_1_stats"
val checkpointLocation = "/tmp"
val triggerTime = "1 minute"
val deduplicateWindow = "1 minute"

## Define The Required Schema and Classes

In [ ]:
// This will be used to give the source `readings_prepared` stream data a schema
val mySchema = StructType(Seq(
    StructField("message_id", StringType, false),
    StructField("reading_ts", TimestampType, false),
    StructField("reading_value", FloatType, false),
    StructField("reading_type", IntegerType, false),
    StructField("plug_id", IntegerType, false),
    StructField("household_id", IntegerType, false),
    StructField("house_id", IntegerType, false)
))

In [ ]:
// Somehow case classes need to be defined in a separate cell
// This will be used in the stateful streaming calculation as the input rows
case class ReadingsInput(
    house_id: Int, 
    hour: Int, 
    reading_value: Float,
    reading_ts: java.sql.Timestamp 
)

// This will be used in the stateful streaming calculation as the state store schema
case class StatsState(
    house_id: Int,
    hour: Int,
    mean: Float,
    m2: Float,
    variance: Float,
    std_dev: Float,
    count: Long, 
    last_ts: java.sql.Timestamp // Will be used in stream-to-stream join
)

### Read and Parse The Input Stream

Only take the Current Load readings (`reading_type = 1`).

In [ ]:
val readings = spark
    .readStream 
    .format("kafka")
    .option("kafka.bootstrap.servers", kafkaBootstrapServer)
    .option("subscribe", kafkaSourceTopic)
    .option("failOnDataLoss", false)
    .load()
    .selectExpr("CAST(value AS STRING)")
    .select(from_json($"value", mySchema).as("data"))
    .select($"data.*")
    .withWatermark("reading_ts", deduplicateWindow) 
    .dropDuplicates()
    .filter($"reading_type" === 1) // Only take the "current load" measurement

#### Peek at The Input Data Stream

In [ ]:
val streamQuery = readings.writeStream.format("memory").queryName("readings").start()

In [ ]:
streamQuery.status

In [ ]:
// Thread.sleep(60000)
spark.sql("select * from readings").show()

In [ ]:
// streamQuery.stop()
streamQuery.lastProgress

## Calculate The Running Average and Standard Deviation

To complete the first task, one thing to keep in mind is that we are calculating a running/moving average/standard deviation over an unbounded stream of data. With that in mind, the typical algorithm of calculating average over a limited amount of bounded/batch data, `average = sum_of_values / population_size`, won't work. 

Why? Since we need to sum all values on a stream, we need to store the state of the sum at every given point in time, and as the stream goes on and one, this can lead to numeric overflow issues. Instead of doing that, we can incrementally calculate average by keeping track of the current average and the current population size / record count, and then adjust as new data comes.  We can do that using [Welford's online algorithm](https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Welford's_online_algorithm).

#### Storing The Running Statistics

Once we have a stream of running average and standard deviation, ideally, I want to output and store the stats in a dataset whose very low latency lookup speed like HBase or Google BigTable. However, due to technical issues with HBase's Spark library, I store the data back to Kafka on the `alert_1_stats` stream. I did not want to spend too much time trying to resolve the issue so I decided to drop it.

This [blog post](https://medium.com/@robbinjain19/challenges-faced-while-integrating-apache-spark-with-hbase-and-the-solution-5c1c8a068808) suggests that I downgrade to Spark 2.1.0 to resolve the HBase connector issue, but I need to use the `foreachBatch` functionality from Spark 2.4.0 and stream-to-stream join from Spark 2.3.0.


### Define the Functions to Be Used in MapGroupsWithState

I created 2 functions: `updateHouseHourStats(state, input)` and `updateAllHouseHourStats(key, input, groupState)`. The latter will be called by Spark's `mapGroupsWithState` function and call the former to work on individual rows to update the state.

The case states "`that household's historical mean consumption for that hour.`", hence here, the grouping keys used are `house_id` and the `hour(reading_ts)`. I looked at the data, the `household_id` is always 0 so it does not make sense to use it.

In [ ]:
/**
 * To be called by updateAllHouseHourStats
 */
def updateHouseHourStats(state: StatsState, input: ReadingsInput): StatsState = {
    // This is implementing Welford's moving average / standard deviation algorithm
    val newCount = state.count + 1
    val delta = input.reading_value - state.mean
    val newMean = state.mean + (delta / newCount)
    val newDelta = input.reading_value - newMean
    val newM2 = state.m2 + (delta * newDelta)
    // Calculate Sample variance, state.count == newCount - 1
    val newVariance = if(newCount > 1) { newM2 / state.count } else { 0 } 
    val newStdDev = if(newCount > 1) { Math.sqrt(newVariance).toFloat } else { 0 }

    return StatsState(
        state.house_id,
        state.hour,
        newMean,
        newM2,
        newVariance,
        newStdDev,
        newCount,
        input.reading_ts
    )
}

/**
 * To be called by mapGroupWithState
 */
def updateAllHouseHourStats(
  key: (Int, Int),
  inputs: Iterator[ReadingsInput],
  groupState: GroupState[StatsState]
) : StatsState = {
    // Get previous state if exists, else create a new empty state
    var currentStatsState = groupState.getOption.getOrElse {
        new StatsState(
            key._1,
            key._2,
            0,  // Mean
            0,  // m2
            0,  // Variance
            0,  // STD Dev
            0,   // Count 
            null
        )
    }
    // Loop over the inputs in this microbatch
    for (input <- inputs) {
        currentStatsState = updateHouseHourStats(currentStatsState, input)
    }
    // Update the current state
    groupState.update(currentStatsState)
    // Return the current state to the stream
    return currentStatsState
}

# Send Update to Kafka

Pull the trigger every arbitrary **1 minute**, as set in `triggerTime`, since we may want to wait until we accumulate enough data for statistics calculation. We are using the `GroupStateTimeout.NoTimeout` option -- meaning that the stats state will continually gets updated until the stream is stopped.

In [ ]:
val stats = readings
    .select($"house_id", hour($"reading_ts").alias("hour"), $"reading_value", $"reading_ts")
    .toDF()
    .as[ReadingsInput]
    .groupByKey(
        input => (input.house_id, input.hour)
    )
    .mapGroupsWithState(GroupStateTimeout.NoTimeout)(updateAllHouseHourStats)
    
val query = stats
    .selectExpr("CONCAT(house_id, '-', hour, '-', last_ts) AS key", "CAST(to_json(struct(*)) AS STRING) AS value")
    .writeStream
    .outputMode("update")
    .format("kafka")
    .trigger(Trigger.ProcessingTime(triggerTime))
    .option("kafka.bootstrap.servers", kafkaBootstrapServer)
    .option("checkpointLocation", checkpointLocation)
    .option("topic", kafkaTargetTopic)
    .start()

In [ ]:
// Thread.sleep(10000)
query.status

In [ ]:
// query.stop()
query.lastProgress